In [1]:
import os
from pprint import pprint
import requests
import pandas as pd
import numpy as np
import time
import csv
import json

In [3]:
# pandora.csv file is taken from the software Pandora, Dario helped me to understand how the application
# works and I took the list of apache projects that at some point used sonarQube.
repos_github = pd.read_csv("/home/mikel/Desktop/project1/cleanRepos070223.csv")

In [50]:
github_fullNames = list(repos_github['full_name'])

'apache/tapestry3'

In [51]:
def getProjectId(projectfullname):

    name = projectfullname + ".csv"

    projectFile = pd.read_csv('/home/mikel/Desktop/project1/' + name)

    return projectFile['project_id'][0]

In [52]:
personal_token = ""
token = os.getenv('GITHUB_TOKEN', personal_token)
headers = {'Authorization': f'token {token}'}

def makeIssueTable(projectFullName,tableToUpdate):

    theIssueQuery = f"https://api.github.com/repos/{projectFullName}/issues"
    params = {'per_page': 100, 'state': 'all'}
    page = 1

    projectissuedataitems = []
    issue_feature_list = ['project_id', 'issue_id', 'number', 'user_login', 'user_type', 'state', 'locked', 'created_at', 'updated_at', 'closed_at', 'active_time', 'title', 'body', 'milestone_url', 'comments', 'pull_request']

    with open(f"/home/mikel/Desktop/project1/{tableToUpdate}", 'w', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow([x for x in issue_feature_list])
        file.close()

    while 1 == 1:
        params['page'] = page
        print(projectFullName+' '+'page '+str(page))
        theResult = requests.get(theIssueQuery, headers=headers, params=params)
        theItemListPerPage = theResult.json()

        project_id = getProjectId(projectFullName)

        if len(theItemListPerPage) == 0: # If there's nothing in request.
            break
        else:
            for item in theItemListPerPage:
                issueItem = {}
                issueItem['project_id'] = project_id
                issueItem['issue_id'] = item['id']
                issueItem['number'] = item['number']
                issueItem['user_login'] = item['user']['login']
                issueItem['user_type'] = item['user']['type']
                issueItem['state'] = item['state']
                issueItem['locked'] = item['locked']

                # pd.to_datetime() -> converts a scalar, array-like, Series or DataFrame/dict-like to a pandas datetime object. (utc -> TRUE the function always returns a timezone-aware UTC-localized Timestamp)
                # pd.timestamp() -> It’s the type used for the entries that make up a DatetimeIndex, and other timeseries oriented data structures in pandas.
                # pd.tz_localize() -> Localize tz-naive index of a Series or DataFrame to target time zone.
                # // -> Floor division
                # pd.Timedelta() -> Represents a duration, the difference between two dates or times.
                issueItem['created_at'] = (pd.to_datetime(item['created_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if item['updated_at'] == None:
                    issueItem['updated_at'] = np.NaN
                else:
                    issueItem['updated_at'] = (pd.to_datetime(item['updated_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if item['closed_at'] == None:
                    issueItem['closed_at'] = np.NaN
                else:
                    issueItem['closed_at'] = (pd.to_datetime(item['closed_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if issueItem['state'] == 'closed':
                    issueItem['active_time'] = issueItem['closed_at'] - issueItem['created_at']
                else:
                    if not issueItem['updated_at']:
                        issueItem['active_time'] = 0
                    else:
                        issueItem['active_time'] = issueItem['updated_at'] - issueItem['created_at']
                issueItem['title'] = item['title']
                issueItem['body'] = item['body']
                if item['milestone'] == None:
                    issueItem['milestone_url'] = False
                else:
                    issueItem['milestone_url'] = True
                issueItem['comments'] = item['comments']
                if 'pull_request' in list(item.keys()):
                    issueItem['pull_request'] = True
                else:
                    issueItem['pull_request'] = False
                projectissuedataitems.append(issueItem)
                with open(f"/home/mikel/Desktop/project1/{tableToUpdate}", 'a', encoding='utf-8') as file:
                    writer = csv.writer(file, delimiter=',')
                    writer.writerow([issueItem[x] for x in issue_feature_list])
            page += 1

In [54]:
for i in range(len(github_fullNames)):

    tableToUpdate = f"githubIssueTables/{github_fullNames[i][7:]}.csv"
    makeIssueTable(github_fullNames[i],tableToUpdate)

apache/tapestry3 page 1
apache/tapestry3 page 2
apache/apr-iconv page 1
apache/apr-iconv page 2
apache/tapestry4 page 1
apache/tapestry4 page 2
apache/sling-old-svn-mirror page 1
apache/sling-old-svn-mirror page 2
apache/sling-old-svn-mirror page 3
apache/sling-old-svn-mirror page 4
apache/xalan-j page 1
apache/xalan-j page 2
apache/apr page 1
apache/apr page 2
apache/stdcxx page 1
apache/zookeeper page 1
apache/zookeeper page 2
apache/zookeeper page 3
apache/zookeeper page 4
apache/zookeeper page 5
apache/zookeeper page 6
apache/zookeeper page 7
apache/zookeeper page 8
apache/zookeeper page 9
apache/zookeeper page 10
apache/zookeeper page 11
apache/zookeeper page 12
apache/zookeeper page 13
apache/zookeeper page 14
apache/zookeeper page 15
apache/zookeeper page 16
apache/zookeeper page 17
apache/zookeeper page 18
apache/zookeeper page 19
apache/zookeeper page 20
apache/zookeeper page 21
apache/lucenenet page 1
apache/lucenenet page 2
apache/lucenenet page 3
apache/lucenenet page 4
apa

## ALTERNATIVE WAY to show the current request rate of my account

In [16]:
headers = {
    'Authorization': 'token TOKEN',
}

response = requests.get('https://api.github.com/rate_limit', headers=headers)

print(response.text)

[{'url': 'https://api.github.com/repos/apache/tapestry3/issues/4',
  'repository_url': 'https://api.github.com/repos/apache/tapestry3',
  'labels_url': 'https://api.github.com/repos/apache/tapestry3/issues/4/labels{/name}',
  'comments_url': 'https://api.github.com/repos/apache/tapestry3/issues/4/comments',
  'events_url': 'https://api.github.com/repos/apache/tapestry3/issues/4/events',
  'html_url': 'https://github.com/apache/tapestry3/pull/4',
  'id': 1395417829,
  'node_id': 'PR_kwDOAAJ02s5AF2v1',
  'number': 4,
  'title': '[SECURITY] Use HTTPS to resolve dependencies in Gradle Build\n',
  'user': {'login': 'JLLeitschuh',
   'id': 1323708,
   'node_id': 'MDQ6VXNlcjEzMjM3MDg=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/1323708?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/JLLeitschuh',
   'html_url': 'https://github.com/JLLeitschuh',
   'followers_url': 'https://api.github.com/users/JLLeitschuh/followers',
   'following_url': 'https://api.github.